## Sparkify Analytics Dashboard

This is a sample analytics dashboard for the fictional Sparkify music streaming service. It uses the `create_connection()` and `run_query()` functions found in the support module `etl_utils.py` to create a connection to the Redshift (Postgres) database and run queries on it. Some example queries can be found below.

### Setup
Before running any analytics in this notebook, you will need to run `create_tables.py` and `etl.py` from the command line in order to create and populate the staging tables and the final analytic tables. More information on this process and the project as a whole can be found in the `README.md` file in this directory.

In [1]:
import configparser
import psycopg2
import time
from etl_utils import create_connection, run_query

In [5]:
cursor, connection = create_connection("dwh.cfg")

## Table Stats

In [6]:
print("Overview of Tables:")
run_query(cursor, connection, 'select count(*) from staging_events')
run_query(cursor, connection, 'select count(*) from staging_songs')
run_query(cursor, connection, 'select count(*) from songplays')
run_query(cursor, connection, 'select count(*) from songs')
run_query(cursor, connection, 'select count(*) from users')
run_query(cursor, connection, 'select count(*) from artists')
run_query(cursor, connection, 'select count(*) from time')

Overview of Tables:
Query: select count(*) from staging_events
Result: (8056,)
Execution time: 1.181175947189331 

Query: select count(*) from staging_songs
Result: (14896,)
Execution time: 0.8895919322967529 

Query: select count(*) from songplays
Result: (333,)
Execution time: 0.9529008865356445 

Query: select count(*) from songs
Result: (14896,)
Execution time: 0.9189109802246094 

Query: select count(*) from users
Result: (104,)
Execution time: 0.8727443218231201 

Query: select count(*) from artists
Result: (10025,)
Execution time: 0.9699840545654297 

Query: select count(*) from time
Result: (8023,)
Execution time: 0.9299471378326416 



## Example Queries

### How many songs were started by free vs paid users?

In [7]:
run_query(cursor, connection, 'select count(songplay_id), users.level from songplays inner join users on songplays.user_id = users.user_id group by 2;')

Query: select count(songplay_id), users.level from songplays inner join users on songplays.user_id = users.user_id group by 2;
Result: (201, 'free')
(276, 'paid')
Execution time: 1.2340350151062012 



### Which hour of the day are users starting to play the most songs?

In [8]:
run_query(cursor, connection, 'select hour, count(songplay_id) from songplays inner join time on songplays.start_time = time.start_time group by hour order by 1 desc;')

Query: select hour, count(songplay_id) from songplays inner join time on songplays.start_time = time.start_time group by hour order by 1 desc;
Result: (23, 11)
(22, 7)
(21, 12)
(20, 18)
(19, 16)
(18, 26)
(17, 40)
(16, 24)
(15, 25)
(14, 16)
(13, 14)
(12, 12)
(11, 16)
(10, 11)
(9, 9)
(8, 18)
(7, 13)
(6, 9)
(5, 7)
(4, 7)
(3, 2)
(2, 3)
(1, 11)
(0, 6)
Execution time: 1.22245192527771 



### Which 30 users have started listening to the most songs?

In [9]:
run_query(cursor, connection, 'select user_id, count(songplay_id) from songplays group by user_id order by 2 desc limit 30;')

Query: select user_id, count(songplay_id) from songplays group by user_id order by 2 desc limit 30;
Result: (49, 42)
(97, 32)
(80, 31)
(44, 21)
(73, 18)
(88, 17)
(15, 15)
(29, 13)
(24, 13)
(36, 13)
(25, 10)
(58, 9)
(95, 8)
(30, 8)
(85, 7)
(16, 6)
(42, 6)
(50, 5)
(8, 3)
(82, 3)
(12, 2)
(2, 2)
(100, 2)
(67, 2)
(66, 2)
(52, 2)
(86, 2)
(72, 2)
(101, 2)
(53, 2)
Execution time: 1.036679983139038 



### What are the most popular operating systems among users who have played songs?

In [10]:
run_query(cursor, connection, """
select 
sum(case when user_agent like '%Windows%' then 1 else 0 end) as windows_sum, 
sum(case when user_agent like '%Linux%' then 1 else 0 end) as linux_sum, 
sum(case when user_agent like '%Mac%' then 1 else 0 end) as mac_sum, 
sum(case when user_agent like '%iPhone%' then 1 else 0 end) as iphone_sum, 
sum(case when user_agent like '%Android%' then 1 else 0 end) as anrdoid_sum 
from songplays;
""")

Query: 
select 
sum(case when user_agent like '%Windows%' then 1 else 0 end) as windows_sum, 
sum(case when user_agent like '%Linux%' then 1 else 0 end) as linux_sum, 
sum(case when user_agent like '%Mac%' then 1 else 0 end) as mac_sum, 
sum(case when user_agent like '%iPhone%' then 1 else 0 end) as iphone_sum, 
sum(case when user_agent like '%Android%' then 1 else 0 end) as anrdoid_sum 
from songplays;

Result: (132, 50, 151, 10, 0)
Execution time: 1.1723508834838867 



### What are the most popular 90s songs among users?

In [11]:
run_query(cursor, connection,
"""
select
    count(songs.song_id) as count,
    songs.title, 
    songs.year
from songplays

join songs on songplays.song_id = songs.song_id

where songs.year BETWEEN 1990 and 1999

group by songs.song_id, songs.title,  songs.year 
order by count(songs.song_id) desc;
""")

Query: 
select
    count(songs.song_id) as count,
    songs.title, 
    songs.year
from songplays

join songs on songplays.song_id = songs.song_id

where songs.year BETWEEN 1990 and 1999

group by songs.song_id, songs.title,  songs.year 
order by count(songs.song_id) desc;

Result: (37, "You're The One", 1990)
(4, 'Mr. Jones', 1991)
(3, 'Pienso En Ti', 1995)
(2, 'God Smack', 1992)
(2, 'Bitter Sweet Symphony', 1997)
(2, 'Lovefool', 1996)
(2, 'You Get What You Give', 1998)
(2, "Don't Panic", 1999)
(1, 'Macarena', 1993)
(1, 'Crazy', 1993)
(1, 'Not For You', 1994)
(1, 'Atrevido', 1999)
(1, 'The Vision', 1997)
(1, 'Star Eyes', 1996)
(1, 'Paradigm Shift', 1998)
(1, 'Tha', 1992)
(1, 'A New Hope', 1997)
(1, 'Lady', 1990)
(1, 'If I Were A Carpenter', 1993)
(1, 'It Was A Good Day', 1992)
(1, 'Va Bien', 1999)
(1, 'Forgotten', 1999)
(1, 'Drifting Away', 1996)
(1, 'Wax on Tha Belt (Baby G Gets Biz)', 1994)
(1, 'The Door', 1997)
(1, 'Ben', 1998)
(1, "The Train Kept A Rollin'", 1991)
(1, 'Engel', 199